In [1]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [2]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [3]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### Logistic Regression

In [4]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets)
train_metric_dict, val_metric_dict

({'acc': 0.9515982055236226,
  'sens': 0.9606124306518724,
  'spec': 0.9410549528541012,
  'f1': 0.9553536790378895},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [5]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.9516040469180803,
  'sens': 0.9606232662968099,
  'spec': 0.9410549528541012,
  'f1': 0.9553593077325115},
 {'acc': 0.7595682041216879,
  'sens': 0.7888349514563107,
  'spec': 0.7253371185237757,
  'f1': 0.7796101949025488})

In [6]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [7]:
val_acc, train_acc

([0.7262021589793916,
  0.7340529931305201,
  0.7520444880601898,
  0.7772325809617272,
  0.7386326463853451,
  0.7634936211972522,
  0.7566241413150148,
  0.7595682041216879],
 [0.7583765596523202,
  0.8085424552549184,
  0.863708584513295,
  0.9034067012477218,
  0.9228234964250666,
  0.9415860554231507,
  0.9474508154586663,
  0.9516040469180803])

In [8]:
PCA_components = [30, 33, 35, 37, 40]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [9]:
val_acc, train_acc

([0.7713444553483808,
  0.7808308799476611,
  0.7867190055610075,
  0.7775596990513576,
  0.7739614000654236],
 [0.8947906444226366,
  0.8986751717369971,
  0.9012337025094631,
  0.9019405112388429,
  0.9032197766250759])

In [10]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9014264685265667,
  'sens': 0.9196970353675451,
  'spec': 0.8800567778566359,
  'f1': 0.909579968815136},
 {'acc': 0.78900883218842,
  'sens': 0.8046116504854369,
  'spec': 0.7707594038325053,
  'f1': 0.8043676069153778})

In [11]:
train_metric_dict, val_metric_dict = log_reg_cross(rbps, targets, PCA_components = 35,reg_parameter=100.0)
train_metric_dict, val_metric_dict

({'acc': 0.9012337025094631,
  'sens': 0.9188085124826629,
  'spec': 0.8806777856635912,
  'f1': 0.9093395103432744},
 {'acc': 0.7867190055610075,
  'sens': 0.7985436893203883,
  'spec': 0.772888573456352,
  'f1': 0.8014616321559075})

### k nearest neighbor classifier

In [12]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

In [13]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.977195196037198,
  'sens': 0.9817202669902912,
  'spec': 0.9719025651424515,
  'f1': 0.9789093816515944},
 {'acc': 0.671573438011122,
  'sens': 0.6644417475728155,
  'spec': 0.6799148332150461,
  'f1': 0.6856606136505948})

In [14]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

In [15]:
val_acc, train_acc

([0.6856395158652273,
  0.6719005561007524,
  0.6728819103696434,
  0.6735361465489041,
  0.6751717369970559,
  0.6738632646385345,
  0.6728819103696434,
  0.671573438011122],
 [0.8914493667928408,
  0.9308963035655872,
  0.9610787887284452,
  0.9741459881302864,
  0.9775106313379129,
  0.9763890836020375,
  0.977212720220571,
  0.977195196037198])

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [16]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 25, 
                                              min_samples_split = 4, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9992055703537549,
  'sens': 0.9994798890429958,
  'spec': 0.9988847206732232,
  'f1': 0.9992633357888807},
 {'acc': 0.741249591102388,
  'sens': 0.7742718446601942,
  'spec': 0.7026259758694109,
  'f1': 0.7633861800777745})

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [17]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.999240618720501,
  'sens': 0.9999241504854369,
  'spec': 0.9984411436682551,
  'f1': 0.9992961254412753},
 {'acc': 0.7190055610075237,
  'sens': 0.7688106796116505,
  'spec': 0.6607523066004258,
  'f1': 0.7468317123489536})

In [18]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]

### SVM

In [19]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',
                                               reg_parameter=0.1, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9073496425066592,
  'sens': 0.9293299237170597,
  'spec': 0.8816409814458076,
  'f1': 0.9153596986013353},
 {'acc': 0.802747791952895,
  'sens': 0.8222087378640777,
  'spec': 0.7799858055358411,
  'f1': 0.8179897373981286})

In [20]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=1.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.7809710734146456,
  'sens': 0.9781119972260749,
  'spec': 0.5503903477643719,
  'f1': 0.8280252440008805},
 {'acc': 0.5763820739286882,
  'sens': 0.8276699029126213,
  'spec': 0.28246983676366216,
  'f1': 0.678100919711658})

In [21]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=10.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9251658956025982,
  'sens': 0.9685007801664355,
  'spec': 0.8744803812227517,
  'f1': 0.933127319611845},
 {'acc': 0.6499836440955185,
  'sens': 0.7384708737864077,
  'spec': 0.5464868701206529,
  'f1': 0.694634703196347})

In [22]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='poly',
                                               reg_parameter=100.0,degree=2, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9673173980092528,
  'sens': 0.9840174237170597,
  'spec': 0.9477846497009024,
  'f1': 0.9701155319114843},
 {'acc': 0.6643768400392541,
  'sens': 0.7032766990291263,
  'spec': 0.6188786373314408,
  'f1': 0.6931818181818182})

In [23]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9391852423010421,
  'sens': 0.9641773578363384,
  'spec': 0.9099538679914834,
  'f1': 0.9447325812873257},
 {'acc': 0.7579326136735361,
  'sens': 0.7918689320388349,
  'spec': 0.7182398864442867,
  'f1': 0.7791044776119403})

In [24]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=10.0, PCA_components = 35)
train_metric_dict, val_metric_dict

({'acc': 0.9663769335015655,
  'sens': 0.975489771151179,
  'spec': 0.955718341275474,
  'f1': 0.9690217859295617},
 {'acc': 0.7621851488387308,
  'sens': 0.7930825242718447,
  'spec': 0.7260468417317246,
  'f1': 0.7824004788985334})

In [25]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='rbf',reg_parameter=1.0, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9542852469741576,
  'sens': 0.976118238557559,
  'spec': 0.9287488593734158,
  'f1': 0.9583710118408034},
 {'acc': 0.7500817795224076,
  'sens': 0.7858009708737864,
  'spec': 0.7083037615330021,
  'f1': 0.7722122838401908})

In [26]:
train_metric_dict, val_metric_dict = svm_cross(rbps, targets,kernel='linear',
                                               reg_parameter=1.0,degree=2, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9594607224636665,
  'sens': 0.9613492545076283,
  'spec': 0.9572518503497921,
  'f1': 0.9623607510494517},
 {'acc': 0.7360157016683022,
  'sens': 0.7682038834951457,
  'spec': 0.6983676366217175,
  'f1': 0.7583108715184187})

### MLP

In [27]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, 
                                               max_iter = 600, hidden_layer_sizes=(3,1))
train_metric_dict, val_metric_dict

/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib

({'acc': 0.9327597083975887,
  'sens': 0.949245839112344,
  'spec': 0.913477136773801,
  'f1': 0.9383512122494229},
 {'acc': 0.7585868498527969,
  'sens': 0.7724514563106796,
  'spec': 0.7423704755145494,
  'f1': 0.7752740560292327})

In [28]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 0, 
                                               max_iter = 500, hidden_layer_sizes=(10,1))
train_metric_dict, val_metric_dict

/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib

({'acc': 0.9873416982101967,
  'sens': 0.9824029126213593,
  'spec': 0.9931182196086383,
  'f1': 0.9881904139077359},
 {'acc': 0.7602224403009487,
  'sens': 0.7402912621359223,
  'spec': 0.7835344215755855,
  'f1': 0.7689883391112512})

In [29]:
train_metric_dict, val_metric_dict = mlp_cross(rbps, targets, PCA_components = 35, 
                                               alpha = 1.0, max_iter = 500, hidden_layer_sizes=(100,1))
train_metric_dict, val_metric_dict

/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Applications/miniconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


({'acc': 0.9533739894387588,
  'sens': 0.9685007801664355,
  'spec': 0.935681334279631,
  'f1': 0.9572569935312514},
 {'acc': 0.7808308799476611,
  'sens': 0.8094660194174758,
  'spec': 0.7473385379701917,
  'f1': 0.7992810065907728})